# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [2]:
import pandas
import re
import warnings
import numpy
import sklearn.neighbors
import sklearn.tree
import sklearn.model_selection
import scipy.stats
import statistics as st
pandas.options.mode.chained_assignment = None

warnings.simplefilter(action = 'ignore', category = FutureWarning)

# Modify this to point to your data.
unique_data = pandas.read_csv('vsiu2_data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09
0,3,1221 m/s,0.1819,FOOTBALL,"Fremont, Santa Clarita, Fresno",0.4284,Tennis,0.6584,88,"Los Angeles, Santa Ana",0.6785
1,2,1079 m/s,0.5138,volleyball,Los Angeles,0.4327,"running, golf",0.7765,1140,San Francisco,0.8071
2,1,830 m/s,0.5483,volleyball,"Long Beach, San Diego",0.7751,"volleyball, soccer",0.419,1297,Moreno Valley,0.7987
3,2,1173 m/s,0.5561,track & field,"Los Angeles, Moreno Valley, Anaheim",0.5675,baseball,0.434,532,"Chula Vista, Fremont, Bakersfield, San Francis...",1.2501
4,3,1124 m/s,0.14,badminton,"Chula Vista, Santa Clarita, Bakersfield, Stock...",0.5653,tennis,1.0012,654,Riverside,0.469
...,...,...,...,...,...,...,...,...,...,...,...
1060,4,364 m/s,1.2072,soccer,Sacramento,1.0678,"boxing, motor sports",0.3296,154,"Anaheim, San Jose, Oakland",0.3662
1061,4,605 m/s,-0.0503,boxing,Sacramento,0.495,"ice hockey, boxing, basketball",0.8359,234,Anaheim,0.9908
1062,3,495 m/s,0.1385,Football,"Stockton, Fresno, Fremont, San Bernardino",0.7263,tennis,0.9447,-431,"Modesto, Santa Ana",1.3889
1063,2,1015 m/s,0.1512,track & field,"Los Angeles, Anaheim, Oakland",0.27,baseball,0.5995,557,"Fresno, Santa Clarita, Fremont, Bakersfield, C...",0.9459


Don't forget to checkout the column information.

In [3]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1065 non-null   int64 
 1   col_00  1059 non-null   object
 2   col_01  1057 non-null   object
 3   col_02  1056 non-null   object
 4   col_03  1053 non-null   object
 5   col_04  1056 non-null   object
 6   col_05  1053 non-null   object
 7   col_06  1058 non-null   object
 8   col_07  1055 non-null   object
 9   col_08  1054 non-null   object
 10  col_09  1058 non-null   object
dtypes: int64(1), object(10)
memory usage: 91.7+ KB


And any numeric information.

In [4]:
unique_data.describe()

,label
count,1065.000000
mean,2.015962
std,1.401774
min,0.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,4.000000


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [5]:
def one_hot(frame, column_name):
    nframe = frame.copy(deep = True)
    num_rows = len(frame)
    iter = 0
    arb_column_name = column_name
    if column_name == "":
        arb_column_name = "empty_value"
    while (iter < num_rows):
        subject = nframe[column_name][iter]
        if isinstance(subject, str):
            if (subject == '?' or subject == 'nan'):
                iter += 1
                continue
            list_data = subject.split(",")
            for list_item in list_data:
                list_item_formatted = list_item.lower()
                list_item_formatted = re.sub("^[ ]*", "", list_item_formatted)
                list_item_formatted = re.sub("[ ]*$", "", list_item_formatted)
                new_cat = arb_column_name + ": " + list_item_formatted
                if new_cat not in nframe.columns:
                    nframe[new_cat] = 0
                nframe.loc[iter, new_cat] = 1
        iter += 1
    return nframe
    
def clean_data(frame):
    for column in frame.columns:
        if frame[column].dtype == 'int64' or frame[column].dtype == 'float':
            continue
        for item in frame.index:
            frame[column][item] = str(frame[column][item])
            frame[column][item] = frame[column][item].lower()

    # with pandas.option_context('display.max_rows', None,
    #                    'display.max_columns', None,
    #                    'display.precision', 3,):
       
    for col in frame.columns:
        convertible_to_num = 0
        for row in frame.index:
            entry = frame[col][row]
            # frame[col][row] = 9
            if isinstance(entry, str):
                x = re.search(r"[0-9]+\.[0-9]+(?!\))", entry)
                if x is not None:
                    y = re.search(r"[0-9]+\.[0-9 ]+[%]{1}(?!\))", entry)
                    
                    convertible_to_num += 1
                    continue
                z = re.search(r"[0-9,]*[0-9]+(?!\))", entry)
                if z is not None:
                    convertible_to_num += 1
                    continue
        print(f"{convertible_to_num / len(frame)} percent of col {col} is convertible to numbers.")
        if (convertible_to_num / len(frame)) > 0.55:
            for row in frame.index:
                entry = frame[col][row]
                # frame[col][row] = 9
                if isinstance(entry, str):
                    x = re.search(r"[0-9]+\.[0-9]+(?!\))", entry)
                    if x is not None:
                        y = re.search(r"[0-9]+\.[0-9 ]+[%]{1}(?!\))", entry)
                        if y is not None:
                            frame[col][row] = float(x.group()) / 100
                        else:
                            frame[col][row] = float(x.group())
                        continue
                    z = re.search(r"[0-9,]*[0-9]+(?!\))", entry)
                    if z is not None:
                        int_entry = z.group()
                        int_entry = int_entry.replace(",", "")
                        x_percent = re.search(r"[0-9,]*[0-9 ]+[%]{1}(?!\))", entry)
                        if x_percent is not None:
                            frame[col][row] = float(int_entry) / 100
                        else:
                            frame[col][row] = int(int_entry)
                    
                        continue
                    frame[col][row] = numpy.nan

    for col in frame.columns:
        contains_str = 0
        types = {"flt": 0, "int": 0}
        for item in frame.index:
            subject = frame[col][item]
            
            if isinstance(subject, int):
                if numpy.isnan(subject):
                    frame.at[item, col] = -9999
                    continue
                types["int"] += 1
            elif isinstance(subject, float):
                if numpy.isnan(subject):
                    frame.at[item, col] = -9999
                    continue
                types["flt"] += 1
            elif isinstance(subject, str):
                contains_str = 1
                break
        if contains_str:
            frame[col].astype('str')
            continue

        max = "None"
        max_int = 0
        for t in types.keys():
            if types[t] > max_int:
                max = t
                max_int = types[t]
        if max_int != 0:
            if max == "int":
                frame[col] = frame[col].astype('int64')
                print(f"casting {col} to int\n\n")
            if max == "flt":
                frame[col] = frame[col].astype(float)
                print(f"casting {col} to float\n\n")
    for col in frame.columns:
        if frame.dtypes[col] != "int64" and frame.dtypes[col] != "float64":
            frame = one_hot(frame, col)
            frame.pop(col)
    
    return frame

unique_data = clean_data(unique_data)
unique_data

0.0 percent of col label is convertible to numbers.
0.9943661971830986 percent of col col_00 is convertible to numbers.
0.9906103286384976 percent of col col_01 is convertible to numbers.
0.0 percent of col col_02 is convertible to numbers.
0.0 percent of col col_03 is convertible to numbers.
0.9896713615023475 percent of col col_04 is convertible to numbers.
0.0 percent of col col_05 is convertible to numbers.
0.9924882629107982 percent of col col_06 is convertible to numbers.
0.9877934272300469 percent of col col_07 is convertible to numbers.
0.0 percent of col col_08 is convertible to numbers.
0.9915492957746479 percent of col col_09 is convertible to numbers.
casting col_00 to int


casting col_01 to float


casting col_04 to float


casting col_06 to float


casting col_07 to int


casting col_09 to float




,label,col_00,col_01,col_04,col_06,col_07,col_09,col_02: football,col_02: volleyball,col_02: track & field,...,col_08: long beach,col_08: anaheim,col_08: fresno,col_08: san bernardino,col_08: san diego,col_08: san jose,col_08: stockton,col_08: modesto,col_08: oakland,col_08: irvine
0,3,1221,0.1819,0.4284,0.6584,88,0.6785,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1079,0.5138,0.4327,0.7765,1140,0.8071,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,830,0.5483,0.7751,0.4190,1297,0.7987,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1173,0.5561,0.5675,0.4340,532,1.2501,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3,1124,0.1400,0.5653,1.0012,654,0.4690,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,4,364,1.2072,1.0678,0.3296,154,0.3662,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1061,4,605,0.0503,0.4950,0.8359,234,0.9908,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1062,3,495,0.1385,0.7263,0.9447,431,1.3889,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1063,2,1015,0.1512,0.2700,0.5995,557,0.9459,0,0,1,...,0,0,1,0,0,0,0,0,0,0


Now we should also be able to view all the numeric columns.

In [6]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 73 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   label                   1065 non-null   int64  
 1   col_00                  1065 non-null   int64  
 2   col_01                  1065 non-null   float64
 3   col_04                  1065 non-null   float64
 4   col_06                  1065 non-null   float64
 5   col_07                  1065 non-null   int64  
 6   col_09                  1065 non-null   float64
 7   col_02: football        1065 non-null   int64  
 8   col_02: volleyball      1065 non-null   int64  
 9   col_02: track & field   1065 non-null   int64  
 10  col_02: badminton       1065 non-null   int64  
 11  col_02: tennis          1065 non-null   int64  
 12  col_02: soccer          1065 non-null   int64  
 13  col_02: running         1065 non-null   int64  
 14  col_02: boxing          1065 non-null   

In [7]:
unique_data

,label,col_00,col_01,col_04,col_06,col_07,col_09,col_02: football,col_02: volleyball,col_02: track & field,...,col_08: long beach,col_08: anaheim,col_08: fresno,col_08: san bernardino,col_08: san diego,col_08: san jose,col_08: stockton,col_08: modesto,col_08: oakland,col_08: irvine
0,3,1221,0.1819,0.4284,0.6584,88,0.6785,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1079,0.5138,0.4327,0.7765,1140,0.8071,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,830,0.5483,0.7751,0.4190,1297,0.7987,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1173,0.5561,0.5675,0.4340,532,1.2501,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,3,1124,0.1400,0.5653,1.0012,654,0.4690,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,4,364,1.2072,1.0678,0.3296,154,0.3662,0,0,0,...,0,1,0,0,0,1,0,0,1,0
1061,4,605,0.0503,0.4950,0.8359,234,0.9908,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1062,3,495,0.1385,0.7263,0.9447,431,1.3889,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1063,2,1015,0.1512,0.2700,0.5995,557,0.9459,0,0,1,...,0,0,1,0,0,0,0,0,0,0


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [11]:
def create_classifiers():
    return [sklearn.linear_model.LogisticRegression(),
            sklearn.tree.DecisionTreeClassifier(max_depth = 1),
            sklearn.neighbors.KNeighborsClassifier(n_neighbors = 20)]
my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(),
 DecisionTreeClassifier(max_depth=1),
 KNeighborsClassifier(n_neighbors=20)]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [9]:
def cross_fold_validation(classifier, frame, folds):
    frame_new = frame.copy()
    target_data = frame_new["label"]
    score = sklearn.model_selection.cross_val_score(classifier, frame, target_data, cv = folds)
    return list(score)
my_classifiers_scores = []
mean_list = []
std_list = []
model_table = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    print(type(accuracy_scores))
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))
    model_table.append(type(classifier).__name__)
    std_list.append(st.stdev(accuracy_scores))
    mean_list.append(st.mean(accuracy_scores))
data_dict = {"Model": model_table, "Average Accuracy": mean_list, "Average STD": std_list}
df = pandas.DataFrame(data = data_dict)
df

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

<class 'list'>
Classifier: LogisticRegression, Accuracy: [0.9389671361502347, 0.8967136150234741, 0.892018779342723, 0.9577464788732394, 0.9671361502347418].
<class 'list'>
Classifier: DecisionTreeClassifier, Accuracy: [0.4131455399061033, 0.4131455399061033, 0.4131455399061033, 0.4131455399061033, 0.4131455399061033].
<class 'list'>
Classifier: KNeighborsClassifier, Accuracy: [0.40375586854460094, 0.39436619718309857, 0.38967136150234744, 0.3755868544600939, 0.4084507042253521].


,Model,Average Accuracy,Average STD
0,LogisticRegression,0.930516,0.034564
1,DecisionTreeClassifier,0.413146,0.000000
2,KNeighborsClassifier,0.394366,0.012857


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [10]:
def significance_test(a_values, b_values, p_value):
    stats, p_resort = scipy.stats.ttest_ind(a_values, b_values)
    print(p_resort)
    if p_resort < p_value:
        return True
    else:
        return False
for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

6.93037630254222e-10
LogisticRegression vs DecisionTreeClassifier: True
8.736930640793515e-10
LogisticRegression vs KNeighborsClassifier: True
0.011424554377833922
DecisionTreeClassifier vs KNeighborsClassifier: True


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.